# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 11:22AM,258367,22,642553,"NBTY Global, Inc.",Released
1,Feb 28 2023 11:21AM,258366,22,642556,"NBTY Global, Inc.",Released
2,Feb 28 2023 11:21AM,258365,22,642530,"NBTY Global, Inc.",Released
3,Feb 28 2023 11:19AM,258364,10,MSP218971,"Methapharm, Inc.",Released
4,Feb 28 2023 11:19AM,258364,10,MSP218972,"Methapharm, Inc.",Released
5,Feb 28 2023 11:19AM,258364,10,MSP218973,"Methapharm, Inc.",Released
6,Feb 28 2023 11:19AM,258364,10,MSP218974,"Methapharm, Inc.",Released
7,Feb 28 2023 11:19AM,258364,10,MSP218976,"Methapharm, Inc.",Released
8,Feb 28 2023 11:19AM,258364,10,MSP218977,"Methapharm, Inc.",Released
9,Feb 28 2023 11:11AM,258363,10,MSP218966,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,258363,Released,1
48,258364,Released,6
49,258365,Released,1
50,258366,Released,1
51,258367,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258363,NaN,NaN,1.0
258364,NaN,NaN,6.0
258365,NaN,NaN,1.0
258366,NaN,NaN,1.0
258367,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,0.0,2.0,5.0
258227,0.0,0.0,1.0
258234,0.0,0.0,1.0
258245,0.0,1.0,0.0
258251,0.0,1.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,0,2,5
258227,0,0,1
258234,0,0,1
258245,0,1,0
258251,0,1,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,0,2,5
1,258227,0,0,1
2,258234,0,0,1
3,258245,0,1,0
4,258251,0,1,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,,2,5
1,258227,,,1
2,258234,,,1
3,258245,,1,
4,258251,,1,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc."
1,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc."
2,Feb 28 2023 11:21AM,258365,22,"NBTY Global, Inc."
3,Feb 28 2023 11:19AM,258364,10,"Methapharm, Inc."
9,Feb 28 2023 11:11AM,258363,10,Methapharm-G
10,Feb 28 2023 11:06AM,258362,12,Hush Hush
11,Feb 28 2023 11:04AM,258361,10,ISDIN Corporation
14,Feb 28 2023 11:01AM,258357,10,ISDIN Corporation
43,Feb 28 2023 10:59AM,258360,20,"Exact-Rx, Inc."
51,Feb 28 2023 10:55AM,258359,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",,,1
1,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",,,1
2,Feb 28 2023 11:21AM,258365,22,"NBTY Global, Inc.",,,1
3,Feb 28 2023 11:19AM,258364,10,"Methapharm, Inc.",,,6
4,Feb 28 2023 11:11AM,258363,10,Methapharm-G,,,1
5,Feb 28 2023 11:06AM,258362,12,Hush Hush,,,1
6,Feb 28 2023 11:04AM,258361,10,ISDIN Corporation,,,3
7,Feb 28 2023 11:01AM,258357,10,ISDIN Corporation,,,29
8,Feb 28 2023 10:59AM,258360,20,"Exact-Rx, Inc.",,,8
9,Feb 28 2023 10:55AM,258359,10,"Snap Medical Industries, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",1,,
1,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",1,,
2,Feb 28 2023 11:21AM,258365,22,"NBTY Global, Inc.",1,,
3,Feb 28 2023 11:19AM,258364,10,"Methapharm, Inc.",6,,
4,Feb 28 2023 11:11AM,258363,10,Methapharm-G,1,,
5,Feb 28 2023 11:06AM,258362,12,Hush Hush,1,,
6,Feb 28 2023 11:04AM,258361,10,ISDIN Corporation,3,,
7,Feb 28 2023 11:01AM,258357,10,ISDIN Corporation,29,,
8,Feb 28 2023 10:59AM,258360,20,"Exact-Rx, Inc.",8,,
9,Feb 28 2023 10:55AM,258359,10,"Snap Medical Industries, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",1,,
1,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",1,,
2,Feb 28 2023 11:21AM,258365,22,"NBTY Global, Inc.",1,,
3,Feb 28 2023 11:19AM,258364,10,"Methapharm, Inc.",6,,
4,Feb 28 2023 11:11AM,258363,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Feb 28 2023 11:21AM,258365,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Feb 28 2023 11:19AM,258364,10,"Methapharm, Inc.",6.0,NaN,NaN
4,Feb 28 2023 11:11AM,258363,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Feb 28 2023 11:21AM,258365,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Feb 28 2023 11:19AM,258364,10,"Methapharm, Inc.",6.0,0.0,0.0
4,Feb 28 2023 11:11AM,258363,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5941728,327.0,1.0,0.0
12,774949,2.0,1.0,0.0
15,1549659,72.0,26.0,0.0
16,516684,2.0,1.0,0.0
19,1033278,6.0,22.0,1.0
20,258360,8.0,0.0,0.0
21,258234,1.0,0.0,0.0
22,775098,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5941728,327.0,1.0,0.0
1,12,774949,2.0,1.0,0.0
2,15,1549659,72.0,26.0,0.0
3,16,516684,2.0,1.0,0.0
4,19,1033278,6.0,22.0,1.0
5,20,258360,8.0,0.0,0.0
6,21,258234,1.0,0.0,0.0
7,22,775098,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,327.0,1.0,0.0
1,12,2.0,1.0,0.0
2,15,72.0,26.0,0.0
3,16,2.0,1.0,0.0
4,19,6.0,22.0,1.0
5,20,8.0,0.0,0.0
6,21,1.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,327.0
1,12,Released,2.0
2,15,Released,72.0
3,16,Released,2.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Executing,1.0,1.0,26.0,1.0,22.0,0.0,0.0,0.0
Released,327.0,2.0,72.0,2.0,6.0,8.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,1.0,1.0,26.0,1.0,22.0,0.0,0.0,0.0
2,Released,327.0,2.0,72.0,2.0,6.0,8.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,1.0,1.0,26.0,1.0,22.0,0.0,0.0,0.0
2,Released,327.0,2.0,72.0,2.0,6.0,8.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()